In [1]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn import preprocessing

from tqdm import tqdm

In [39]:
ksp = pd.read_csv('ksp_final.csv')
ksp

,name,매출액(억원),영업이익(억원),당기순이익(억원),영업이익률(%),순이익률(%),ROE(%),부채비율(%),당좌비율(%),유보율(%),...,BPS(원),PBR(배),주당배당금(원),시가배당률(%),배당성향(%),자기자본이익률(%),주당순이익(원),주가수익비율(배),주당순자산가치(원),주가순자산비율(배)
0,삼성전자,3054876,463279,371012,15.17,12.14,11.67,39.92(21),196.75(21),"33,143.62(21)",...,48462,1.14,1521,1.84(21),25.00(21),11.67,5374,5374,48462,1.14
1,LG에너지솔루션,251891,14273,9004,5.67,3.57,6.5,171.83(21),59.53(21),"7,460.02(21)",...,76516,5.8,-,-,-,6.5,3622,3622,76516,5.8
2,SK하이닉스,450663,75122,50179,16.67,11.13,7.51,54.98(21),99.92(21),"1,643.64(21)",...,96591,0.95,1446,1.18(21),11.03(21),7.51,6636,6636,96591,0.95
3,삼성바이오로직스,27487,9093,6102,33.08,22.2,10.26,59.68(21),80.95(21),"2,922.69(21)",...,98707,8.1,-,-,-,10.26,8815,8815,98707,8.1
4,LG화학,527186,33343,27163,6.33,5.15,9.32,120.32(21),80.50(21),"5,311.19(21)",...,389904,1.76,10375,1.95(21),25.49(21),9.32,31091,31091,389904,1.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,쿠쿠 홈시스(쿠쿠홈시스),"8,932(21)","1,786(21)","1,692(21)",20.00(21),18.94(21),26.53(21),38.55(21),173.48(21),"28,714.40(21)",...,"28,790(21)",1.36(21),650(21),1.66(21),9.58(21),26.53(21),"6,780(21)","6,780(21)","28,790(21)",1.36(21)
596,에이치디씨현대산업개발(HDC현대산업개발),"33,639(21)","2,734(21)","1,763(21)",8.13(21),5.24(21),6.28(21),127.58(21),143.88(21),776.48(21),...,"43,665(21)",0.52(21),600(21),2.62(21),22.40(21),6.28(21),"2,679(21)","2,679(21)","43,665(21)",0.52(21)
597,HDC 현대산업(HDC현대산업개발),"33,639(21)","2,734(21)","1,763(21)",8.13(21),5.24(21),6.28(21),127.58(21),143.88(21),776.48(21),...,"43,665(21)",0.52(21),600(21),2.62(21),22.40(21),6.28(21),"2,679(21)","2,679(21)","43,665(21)",0.52(21)
598,한 섬(한섬),15672,1712,1257,10.92,8.02,9.97,25.53(21),81.15(21),"10,157.70(21)",...,61557,0.45,625,1.69(21),11.51(21),9.97,5242,5242,61557,0.45


In [40]:
ksp['주가수익비율(배)'] = ksp['PER(배)']

In [41]:
col = ksp.columns
lst = []
for i in col :
    if '원' in i :
        lst.append(i)



for i in range(ksp.shape[0]):
    for c in lst :
        if '-' in ksp.loc[i,c]:
            continue
        if '(' not in ksp.loc[i,c] :
            ksp.loc[i,c] = int(ksp.loc[i,c])
            ksp.loc[i,c] = format(ksp.loc[i,c], ',d')   

In [43]:
list(ksp.iloc[158]) 
ksp.iloc[158]= ['강원랜드',
 '12,962 ',
 '2,587 ',
 '1,607 ',
 '20 ',
 '12 ',
 '5 ',
 '17.66(21)',
 '270.31(21)',
 '3,103.60(21)',
 '723 ',
 '32 ',
 '16,698 ',
 '1 ',
 '472 ',
 '-',
 '-',
 '5 ',
 '723 ',
 '723 ',
 '16,698 ',
 '1 ']

#강원 랜드(강원랜드) kangwonland(강원랜드)

a = list(ksp.iloc[158][1:])
a.insert(0,'강원 랜드(강원랜드)')
ksp.iloc[216] = a
a = list(ksp.iloc[158][1:])
a.insert(0,'kangwonland(강원랜드)')
ksp.iloc[217] = a

# a.insert(0,'kangwonland(강원랜드)')

list(ksp.iloc[190]) #160 218 219 / 192 282 283
ksp.iloc[190] = ['현대미포조선',
 '36494',
 '-374',
 '927',
 '-1.03',
 '2.54',
 '4.29',
 '75.94(21)',
 '54.62(21)',
 '859.07(21)',
 '2381',
 '33.93',
 '56853',
 '1.42',
 '62',
 '-',
 '-',
 '4.29',
 '2381',
 '2381',
 '56853',
 '1.42']

a = list(ksp.iloc[190][1:])
a.insert(0,'현미포(현대미포조선)')
ksp.iloc[280] = a
a = list(ksp.iloc[190][1:])
a.insert(0,'현대미포(현대미포조선)')
ksp.iloc[281] = a

# a.insert(0,'kangwonland(강원랜드)')

ksp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        600 non-null    object
 1   매출액(억원)     600 non-null    object
 2   영업이익(억원)    600 non-null    object
 3   당기순이익(억원)   600 non-null    object
 4   영업이익률(%)    600 non-null    object
 5   순이익률(%)     600 non-null    object
 6   ROE(%)      600 non-null    object
 7   부채비율(%)     600 non-null    object
 8   당좌비율(%)     600 non-null    object
 9   유보율(%)      600 non-null    object
 10  EPS(원)      600 non-null    object
 11  PER(배)      600 non-null    object
 12  BPS(원)      600 non-null    object
 13  PBR(배)      600 non-null    object
 14  주당배당금(원)    600 non-null    object
 15  시가배당률(%)    600 non-null    object
 16  배당성향(%)     600 non-null    object
 17  자기자본이익률(%)  600 non-null    object
 18  주당순이익(원)    600 non-null    object
 19  주가수익비율(배)   600 non-null    object
 20  주당순자산가치(원)

In [44]:
ksp_origin = ksp.copy()
ksp_origin = ksp_origin.iloc[:,:-5]


question = ['알려줘', '궁금해','얼마야','어때','말해봐','괜찮아']

Q = []
A = []
for k in range(ksp_origin.shape[0]) :
    if '(' in ksp_origin['name'][k]:
        name = ksp_origin['name'][k].split('(')[0]
        name2 = ksp_origin['name'][k].split('(')[1].split(")")[0]
    else: 
        name = ksp_origin['name'][k]
        name2 = ksp_origin['name'][k]        
    for j in ksp_origin.columns[1:] :
        col = j.split('(')[0]
        cnt = j.split('(')[1].split(')')[0]
        Q.append(f'{name} {col}') 
        Q.append(f'{name}의 최근 {col}')
        for z in question:
            Q.append(f'{name} {col} {z}') #6
        if '(' in ksp_origin[j][k]:
            for _ in range(8):
                if col == 'ROE' or col == 'EPS':
                    A.append(f'{name2}의 2021년도 {col}는 {ksp_origin[j][k][:-4]}({cnt})입니다.')
                else:
                    A.append(f'{name2}의 2021년도 {col}은 {ksp_origin[j][k][:-4]}({cnt})입니다.')
        else:
            for _ in range(8):
                if col == 'ROE' or col == 'EPS':
                    A.append(f'{name2}의 2022년도 {col}는 {ksp_origin[j][k]}({cnt})입니다.')
                else:
                    A.append(f'{name2}의 2022년도 {col}은 {ksp_origin[j][k]}({cnt})입니다.')
test = pd.DataFrame({'Q':Q,'A':A})
test

,Q,A
0,삼성전자 매출액,"삼성전자의 2022년도 매출액은 3,054,876(억원)입니다."
1,삼성전자의 최근 매출액,"삼성전자의 2022년도 매출액은 3,054,876(억원)입니다."
2,삼성전자 매출액 알려줘,"삼성전자의 2022년도 매출액은 3,054,876(억원)입니다."
3,삼성전자 매출액 궁금해,"삼성전자의 2022년도 매출액은 3,054,876(억원)입니다."
4,삼성전자 매출액 얼마야,"삼성전자의 2022년도 매출액은 3,054,876(억원)입니다."
...,...,...
76795,HANDSOME 배당성향 궁금해,한섬의 2021년도 배당성향은 11.51(%)입니다.
76796,HANDSOME 배당성향 얼마야,한섬의 2021년도 배당성향은 11.51(%)입니다.
76797,HANDSOME 배당성향 어때,한섬의 2021년도 배당성향은 11.51(%)입니다.
76798,HANDSOME 배당성향 말해봐,한섬의 2021년도 배당성향은 11.51(%)입니다.


<!-- 자기자본이익률(%)	주당순이익(원)	주가수익비율(배)	주당순자산가치(원)	주가순자산비율(배) -->

In [45]:
lst = ['ROE(%)','EPS(원)','PER(배)','BPS(원)','PBR(배)']

ksp_add = ksp.iloc[:,-5:]
# ksp_add
ksp_add['name'] = ksp['name']


question = ['알려줘', '궁금해','얼마야','어때','말해봐','괜찮아']

Q_2 = []
A_2 = []
for k in range(ksp_add.shape[0]) :
    if '(' in ksp_add['name'][k]:
        name = ksp_add['name'][k].split('(')[0]
        name2 = ksp_origin['name'][k].split('(')[1].split(")")[0]
    else: 
        name = ksp_add['name'][k]
        name2 = ksp_add['name'][k]
    for j,ls in zip(ksp_add.columns[:-1], lst):
        col = j.split('(')[0]
        lsl = ls.split('(')[0]
        cnt = j.split('(')[1].split(')')[0]
        Q_2.append(f'{name} {col}') 
        Q_2.append(f'{name}의 최근 {col}')
        for z in question:
            Q_2.append(f'{name} {col} {z}') #6
        if '(' in ksp_add[j][k]:
            for _ in range(8):
                if lsl == 'ROE' or lsl == 'EPS':
                    A_2.append(f'{name2}의 2021년도 {lsl}는 {ksp_add[j][k][:-4]}({cnt})입니다.')
                else:
                    A_2.append(f'{name2}의 2021년도 {lsl}은 {ksp_add[j][k][:-4]}({cnt})입니다.')
        else:
            for _ in range(8):
                if lsl == 'ROE' or lsl == 'EPS':
                    A_2.append(f'{name2}의 2022년도 {lsl}는 {ksp_add[j][k]}({cnt})입니다.')
                else:
                    A_2.append(f'{name2}의 2022년도 {lsl}은 {ksp_add[j][k]}({cnt})입니다.')
test_2 = pd.DataFrame({'Q':Q_2,'A':A_2})
test_2

,Q,A
0,삼성전자 자기자본이익률,삼성전자의 2022년도 ROE는 11.67(%)입니다.
1,삼성전자의 최근 자기자본이익률,삼성전자의 2022년도 ROE는 11.67(%)입니다.
2,삼성전자 자기자본이익률 알려줘,삼성전자의 2022년도 ROE는 11.67(%)입니다.
3,삼성전자 자기자본이익률 궁금해,삼성전자의 2022년도 ROE는 11.67(%)입니다.
4,삼성전자 자기자본이익률 얼마야,삼성전자의 2022년도 ROE는 11.67(%)입니다.
...,...,...
23995,HANDSOME 주가순자산비율 궁금해,한섬의 2022년도 PBR은 0.45(배)입니다.
23996,HANDSOME 주가순자산비율 얼마야,한섬의 2022년도 PBR은 0.45(배)입니다.
23997,HANDSOME 주가순자산비율 어때,한섬의 2022년도 PBR은 0.45(배)입니다.
23998,HANDSOME 주가순자산비율 말해봐,한섬의 2022년도 PBR은 0.45(배)입니다.


In [46]:
final = pd.concat([test,test_2])
final

,Q,A
0,삼성전자 매출액,"삼성전자의 2022년도 매출액은 3,054,876(억원)입니다."
1,삼성전자의 최근 매출액,"삼성전자의 2022년도 매출액은 3,054,876(억원)입니다."
2,삼성전자 매출액 알려줘,"삼성전자의 2022년도 매출액은 3,054,876(억원)입니다."
3,삼성전자 매출액 궁금해,"삼성전자의 2022년도 매출액은 3,054,876(억원)입니다."
4,삼성전자 매출액 얼마야,"삼성전자의 2022년도 매출액은 3,054,876(억원)입니다."
...,...,...
23995,HANDSOME 주가순자산비율 궁금해,한섬의 2022년도 PBR은 0.45(배)입니다.
23996,HANDSOME 주가순자산비율 얼마야,한섬의 2022년도 PBR은 0.45(배)입니다.
23997,HANDSOME 주가순자산비율 어때,한섬의 2022년도 PBR은 0.45(배)입니다.
23998,HANDSOME 주가순자산비율 말해봐,한섬의 2022년도 PBR은 0.45(배)입니다.


In [49]:
final.to_csv('chatbotdata_final.csv',index=False)

In [50]:
df = pd.read_csv('./chatbotdata_final.csv')
df['intent'] =df['A'].str.split().str.get(0).str.slice(0,-1)+'_'+df['A'].str.split().str.get(2).str.slice(0,-1)
df_sort = df.sort_values(by='A')
df_sort = df_sort.reset_index(drop=True)
df_sort.to_csv('qa_final.csv',index=False)

In [95]:
dt = pd.read_csv('qa_final.csv')
dt

,Q,A,intent
0,BGF리테일 당좌비율 궁금해,BGF리테일의 2021년도 당좌비율은 68.85(%)입니다.,BGF리테일_당좌비율
1,비지에프리테일 당좌비율 알려줘,BGF리테일의 2021년도 당좌비율은 68.85(%)입니다.,BGF리테일_당좌비율
2,비지에프리테일 당좌비율 궁금해,BGF리테일의 2021년도 당좌비율은 68.85(%)입니다.,BGF리테일_당좌비율
3,비지에프리테일 당좌비율 얼마야,BGF리테일의 2021년도 당좌비율은 68.85(%)입니다.,BGF리테일_당좌비율
4,비지에프리테일 당좌비율 어때,BGF리테일의 2021년도 당좌비율은 68.85(%)입니다.,BGF리테일_당좌비율
...,...,...,...
100795,휠라홀딩 주당배당금 얼마야,"휠라홀딩스의 2022년도 주당배당금은 1,317(원)입니다.",휠라홀딩스_주당배당금
100796,휠라홀딩 주당배당금 궁금해,"휠라홀딩스의 2022년도 주당배당금은 1,317(원)입니다.",휠라홀딩스_주당배당금
100797,휠라홀딩 주당배당금 알려줘,"휠라홀딩스의 2022년도 주당배당금은 1,317(원)입니다.",휠라홀딩스_주당배당금
100798,휠라홀딩스 주당배당금 어때,"휠라홀딩스의 2022년도 주당배당금은 1,317(원)입니다.",휠라홀딩스_주당배당금


In [96]:
def word_clean(df):
    from hanspell import spell_checker
    import re
    from tqdm import tqdm
    CHANGE_FILTER = re.compile("([~!?\"':;&)(])") #제외할 문자
    okt = Okt()
    X_train = []
    for sentence in tqdm(df,desc='전처리 중'):
        sentence = re.sub(CHANGE_FILTER, "", sentence) #문자 제외
        ok = spell_checker.check(sentence) # 맞춤법 확인
        sentence = ok.checked #바꾼 문자로 변경
        clean_words = []
        for word in okt.pos(sentence):
            if word[1] in ['Noun', 'Modifier', 'Alpha']: #명사, 관형사(가끔 나와서), 영어
                 clean_words.append(word[0]) #찾아진 것 중 단어만 추가
        sentence = ' '.join(clean_words)
        X_train.append(sentence)
    return X_train

In [98]:
X_train_sort = word_clean(dt.Q)

전처리 중: 100%|██████████| 100800/100800 [48:37<00:00, 34.55it/s] 


In [99]:
X_train_sort

['BGF 리테일 당좌 비율',
 '비지 프 리테일 당좌 비율',
 '비지 프 리테일 당좌 비율',
 '비지 프 리테일 당좌 비율 얼마',
 '비지 프 리테일 당좌 비율',
 '비지 프 리테일 당좌 비율 말',
 '비지 프 리테일 당좌 비율',
 '비지 엪 리테일 당좌 비율',
 '비지 엪 리테일 최근 당좌 비율',
 '비지 엪 리테일 당좌 비율',
 '비지 엪 리테일 당좌 비율',
 '비지 엪 리테일 당좌 비율 얼마',
 '비지 엪 리테일 당좌 비율',
 '비지 엪 리테일 당좌 비율 말',
 '비지 프 리테일 최근 당좌 비율',
 'BGF 리테일 당좌 비율',
 '비지 엪 리테일 당좌 비율',
 '비지 프 리테일 당좌 비율',
 'BGF 리테일 최근 당좌 비율',
 'BGF 리테일 당좌 비율',
 'BGF 리테일 당좌 비율 얼마',
 'BGF 리테일 당좌 비율',
 'BGF 리테일 당좌 비율 말',
 'BGF 리테일 당좌 비율',
 '비지 프 리테일 배당 성향',
 '비지 프 리테일 최근 배당 성향',
 '비지 프 리테일 배당 성향',
 '비지 프 리테일 배당 성향',
 '비지 프 리테일 배당 성향 얼마',
 '비지 프 리테일 배당 성향',
 '비지 프 리테일 배당 성향 말',
 '비지 엪 리테일 배당 성향',
 'BGF 리테일 배당 성향',
 'BGF 리테일 최근 배당 성향',
 'BGF 리테일 배당 성향',
 'BGF 리테일 배당 성향',
 'BGF 리테일 배당 성향 얼마',
 'BGF 리테일 배당 성향',
 'BGF 리테일 배당 성향 말',
 'BGF 리테일 배당 성향',
 '비지 엪 리테일 배당 성향',
 '비지 엪 리테일 최근 배당 성향',
 '비지 엪 리테일 배당 성향',
 '비지 엪 리테일 배당 성향',
 '비지 엪 리테일 배당 성향',
 '비지 엪 리테일 배당 성향 말',
 '비지 프 리테일 배당 성향',
 '비지 엪 리테일 배당 성향 얼마',
 '비지 프 리테일 부채 비율',
 'BGF 리테일 부채 비율',
 'BGF 리테일 부채 비율 말',

In [100]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train_sort)

X_train_sort = tokenizer.texts_to_sequences(X_train_sort)

In [101]:
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(df['A'])

y_train = idx_encode.transform(df['A'])

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))

In [102]:
idx_label = {}
for key, value in label_idx.items():
  idx_label[value] = key

In [103]:
idx_label

{0: 'BGF리테일의 2021년도 당좌비율은 68.85(%)입니다.',
 1: 'BGF리테일의 2021년도 배당성향은 35.10(%)입니다.',
 2: 'BGF리테일의 2021년도 부채비율은 220.81(%)입니다.',
 3: 'BGF리테일의 2021년도 시가배당률은 2.06(%)입니다.',
 4: 'BGF리테일의 2021년도 유보율은 4,439.53(%)입니다.',
 5: 'BGF리테일의 2022년도 BPS은 55,268(원)입니다.',
 6: 'BGF리테일의 2022년도 EPS는 11,332(원)입니다.',
 7: 'BGF리테일의 2022년도 PBR은 3.45(배)입니다.',
 8: 'BGF리테일의 2022년도 PER은 16.85(배)입니다.',
 9: 'BGF리테일의 2022년도 ROE는 22.21(%)입니다.',
 10: 'BGF리테일의 2022년도 당기순이익은 1,963(억원)입니다.',
 11: 'BGF리테일의 2022년도 매출액은 76,042(억원)입니다.',
 12: 'BGF리테일의 2022년도 순이익률은 2.58(%)입니다.',
 13: 'BGF리테일의 2022년도 영업이익률은 3.45(%)입니다.',
 14: 'BGF리테일의 2022년도 영업이익은 2,626(억원)입니다.',
 15: 'BGF리테일의 2022년도 주당배당금은 3,307(원)입니다.',
 16: 'BNK금융지주의 2021년도 매출액은 55,807(억원)입니다.',
 17: 'BNK금융지주의 2021년도 배당성향은 22.97(%)입니다.',
 18: 'BNK금융지주의 2021년도 부채비율은 1,154.86(%)입니다.',
 19: 'BNK금융지주의 2021년도 순이익률은 14.95(%)입니다.',
 20: 'BNK금융지주의 2021년도 시가배당률은 6.67(%)입니다.',
 21: 'BNK금융지주의 2021년도 영업이익률은 19.57(%)입니다.',
 22: 'BNK금융지주의 2021년도 유보율은 527.74(%)입니다.',
 23: 'BNK금융지주의 2022년도 BPS은 30

In [104]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)

In [105]:
vocab_size = len(tokenizer.word_index)+1
vocab_size

601

In [61]:
max(len(l) for l in X_train_sort)

13

In [106]:
sum(map(len, X_train_sort))/len(X_train_sort)

4.757083333333333

In [107]:
max_len = 13
X_train_sort= pad_sequences(X_train_sort, maxlen = max_len, padding='post')
X_train_sort[:5]

array([[296,  60,  15,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [165, 166,  60,  15,   2,   0,   0,   0,   0,   0,   0,   0,   0],
       [165, 166,  60,  15,   2,   0,   0,   0,   0,   0,   0,   0,   0],
       [165, 166,  60,  15,   2,   7,   0,   0,   0,   0,   0,   0,   0],
       [165, 166,  60,  15,   2,   0,   0,   0,   0,   0,   0,   0,   0]])

In [108]:
np.save('X_train.npy',X_train_sort)